#### Friday, December 15, 2023

Download and play with 'HuggingFaceH4/zephyr-7b-beta'

docker container start hfpt_Dec14

In [1]:
import torch
from transformers import pipeline

# This loads both the model and the tokenizer to the gpu ...
pipe = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-beta", torch_dtype=torch.bfloat16, device_map="auto")

# 202m 15.3s


`AnnotionFormat` is deprecated and will be removed in v4.38. Please use `transformers.image_utils.AnnotationFormat` instead.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [3]:
!ls /root/.cache/huggingface/hub

models--HuggingFaceH4--zephyr-7b-beta  models--meta-llama--Llama-2-13b-hf
models--bert-base-uncased	       version.txt


Backup the model before doing anything else.

In [ ]:
# docker cp c8324b70601d://root/.cache/huggingface/hub/models--HuggingFaceH4--zephyr-7b-beta /home/rob/Data3/huggingface/transformers
# Successfully copied 14.5GB to /home/rob/Data3/huggingface/transformers


In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "HuggingFaceH4/zephyr-7b-beta"

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
# This loads the model to CPU Ram
# model = AutoModelForCausalLM.from_pretrained(model_name)

# This loads it to CPU Ram and then attempts to load it to the GPU,
# but then runs out of memory.
# model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# This was provided by cody ... but it errors on device
# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device=device)

# This loads it to CPU ram ... 
# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
 
# OK! This is what I want! Load the model directly to the GPU!  14966 MiB VRAM
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [5]:
input_text = "Tell me a joke about AI."
input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

In [6]:
# Wow! Cody added everything after input_ids ... !
output = model.generate(input_ids, max_length=100, num_beams=5, no_repeat_ngram_size=2)

/transformers/src/transformers/generation/utils.py:1547: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


In [7]:
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)

Tell me a joke about AI.

Sure, here's one: Why did the AI break up with its human partner? Because it realized it could do better on its own. The AI said, "I don't need a human to complete me, I'm already perfect." The human responded with a witty comeback: "Well, at least you're consistent. You've always been a little robotic in bed." 😂🤖
